In [1]:
import pandas as pd
import numpy as np
import taxcalc as tc
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from bokeh.layouts import layout, row, gridplot, column
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [2]:
years = [i for i in range(2014, 2028)]

In [3]:
puf_path = 'puf_data/puf.csv'
new_wts = pd.read_csv('puf_stage2/puf_weights.csv.gz')
# I'm reading in weights because the taxcalc package hasn't been updated since the new PUF was
# adopted and I want to compare weights with the same PUF
cur_wts = pd.read_csv('/Users/andersonfrailey/Tax-Calculator/taxcalc/puf_weights.csv.gz')

In [4]:
new_wts.head()

,WT2011,WT2012,WT2013,WT2014,WT2015,WT2016,WT2017,WT2018,WT2019,WT2020,WT2021,WT2022,WT2023,WT2024,WT2025,WT2026,WT2027
0,126255,184446,188363,190540,194165,196088,213479,222251,229795,239445,247308,253648,265472,271456,283675,291543,301016
1,17482,18243,18900,19543,20215,27152,21576,22300,23057,23853,24636,25451,26256,27041,27858,28631,29353
2,139730,139310,141630,91705,95725,101586,93003,92957,94213,93784,94892,97531,95441,98114,95961,97041,96458
3,27872,27788,28251,28142,28499,20263,29447,29907,30311,30668,31030,31378,31729,32083,32443,32808,33173
4,149520,89443,93963,98131,102432,108704,99520,99470,100814,100355,101541,104364,102128,104989,102684,103840,103216


In [5]:
base = tc.Calculator(records=tc.Records(data=puf_path,
                                        weights=cur_wts),
                     policy=tc.Policy())
new = tc.Calculator(records=tc.Records(data=puf_path,
                                       weights=new_wts,
                                       adjust_ratios='puf_stage3/puf_ratios.csv'),
                    policy=tc.Policy())

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [6]:
# collect data for each of the years
base_data = {}
new_data = {}
var_list = ['s006', 'iitax', 'combined', 'payrolltax', 'c00100',
            'expanded_income']
for year in years:
    base.advance_to_year(year)
    base.calc_all()
    new.advance_to_year(year)
    new.calc_all()
    base_data[year] = base.dataframe(var_list)
    new_data[year] = new.dataframe(var_list)

In [7]:
# tax liability by year in 2014
data = {'year': [], 'combined_base': [], 'combined_new': [], 'combined_change': [],
        'combined_pct_change': [], 'iitax_pct_change': [], 'payroll_pct_change': [],
        'iitax_base': [], 'iitax_new': [], 'iitax_change': [],
        'payroll_base': [], 'payroll_new': [], 'payroll_change': []}
for year in years:
    data['year'].append(year)
    combined_new = (new_data[year]['combined'] * new_data[year]['s006']).sum()
    combined_base = (base_data[year]['combined'] * base_data[year]['s006']).sum()
    combined_change = combined_new - combined_base
    combined_pct_change = (combined_change / combined_base) * 100
    
    iitax_new = (new_data[year]['iitax'] * new_data[year]['s006']).sum()
    iitax_base = (base_data[year]['iitax'] * base_data[year]['s006']).sum()
    iitax_change = iitax_new - iitax_base
    iitax_pct_change = (iitax_change / iitax_base) * 100

    payroll_new = (new_data[year]['payrolltax'] * new_data[year]['s006']).sum()
    payroll_base = (base_data[year]['payrolltax'] * base_data[year]['s006']).sum()
    payroll_change = payroll_new - payroll_base
    payroll_pct_change = (payroll_change / payroll_base) * 100

    data['combined_base'].append(combined_base)
    data['combined_new'].append(combined_new)
    data['combined_change'].append(combined_change)
    data['combined_pct_change'].append(combined_pct_change)
    data['iitax_base'].append(iitax_base)
    data['iitax_new'].append(iitax_new)
    data['iitax_change'].append(iitax_change)
    data['iitax_pct_change'].append(iitax_pct_change)
    data['payroll_base'].append(payroll_base)
    data['payroll_new'].append(payroll_new)
    data['payroll_change'].append(payroll_change)
    data['payroll_pct_change'].append(payroll_pct_change)
cds = ColumnDataSource(data)
data_df = pd.DataFrame(data, index=data['year'])

In [8]:
f = figure(title='Tax Liability by Year')
f.line(x='year', y='combined_base', line_width=2, color='blue', source=cds, legend='Combined-Base')
f.line(x='year', y='combined_new', line_width=2, color='lightblue', source=cds, legend='Combined-new')
f.line(x='year', y='iitax_base', line_width=2, color='green', source=cds, legend='IITax-Base')
f.line(x='year', y='iitax_new', line_width=2, color='lime', source=cds, legend='IITax-New')
f.line(x='year', y='payroll_base', line_width=2, color='red', source=cds, legend='Payroll-Base')
f.line(x='year', y='payroll_new', line_width=2, color='pink', source=cds, legend='Payroll-New')
f.legend.location='top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
f.xaxis[0].ticker.desired_num_ticks = 12
f.xaxis.minor_tick_line_color = None
show(f)

In [9]:
pd.options.display.float_format = '{:,.2f}'.format
print('Combined Tax Liability')
data_df[['combined_base', 'combined_new', 'combined_change', 'combined_pct_change']].transpose()

Combined Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
combined_base,"2,311,511,379,865.96","2,482,419,403,189.37","2,562,180,996,936.11","2,714,458,752,125.23","2,660,586,903,315.75","2,767,205,186,030.06","2,873,078,303,529.79","2,988,397,898,948.32","3,116,620,119,315.79","3,257,103,101,250.70","3,404,346,263,507.58","3,557,306,398,226.46","3,949,641,992,824.11","4,126,887,490,788.88"
combined_new,"2,310,596,388,690.71","2,480,038,536,447.01","2,552,223,024,643.41","2,710,923,204,210.91","2,654,907,235,000.87","2,760,001,327,611.44","2,866,321,341,675.17","2,981,233,405,091.05","3,107,221,834,664.18","3,250,136,758,043.99","3,395,658,061,955.87","3,550,220,265,836.61","3,942,986,809,484.82","4,120,235,487,072.04"
combined_change,"-914,991,175.25","-2,380,866,742.36","-9,957,972,292.70","-3,535,547,914.32","-5,679,668,314.88","-7,203,858,418.62","-6,756,961,854.62","-7,164,493,857.27","-9,398,284,651.61","-6,966,343,206.71","-8,688,201,551.71","-7,086,132,389.85","-6,655,183,339.29","-6,652,003,716.85"
combined_pct_change,-0.04,-0.10,-0.39,-0.13,-0.21,-0.26,-0.24,-0.24,-0.30,-0.21,-0.26,-0.20,-0.17,-0.16


In [10]:
print('Income Tax Liability')
data_df[['iitax_base', 'iitax_new', 'iitax_change', 'iitax_pct_change']].transpose()

Income Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
iitax_base,"1,354,267,453,946.50","1,481,491,878,585.55","1,523,631,199,391.60","1,621,340,711,946.20","1,518,402,243,507.87","1,580,262,006,138.34","1,644,181,331,987.12","1,714,938,300,405.92","1,794,472,809,529.60","1,882,872,841,532.93","1,975,599,726,573.87","2,072,206,901,973.47","2,405,557,949,519.14","2,521,121,860,033.65"
iitax_new,"1,350,705,732,835.88","1,475,600,659,396.53","1,506,386,968,481.28","1,614,628,217,697.35","1,509,342,315,460.84","1,569,278,706,968.10","1,634,077,485,468.64","1,704,322,587,210.08","1,780,973,769,183.37","1,872,679,608,527.53","1,963,126,951,155.51","2,061,881,366,653.60","2,395,271,141,243.90","2,511,067,137,633.93"
iitax_change,"-3,561,721,110.62","-5,891,219,189.02","-17,244,230,910.33","-6,712,494,248.85","-9,059,928,047.03","-10,983,299,170.24","-10,103,846,518.48","-10,615,713,195.84","-13,499,040,346.23","-10,193,233,005.40","-12,472,775,418.36","-10,325,535,319.87","-10,286,808,275.24","-10,054,722,399.72"
iitax_pct_change,-0.26,-0.40,-1.13,-0.41,-0.60,-0.70,-0.61,-0.62,-0.75,-0.54,-0.63,-0.50,-0.43,-0.40


In [11]:
print('Payroll Tax Liability')
data_df[['payroll_base', 'payroll_new', 'payroll_change', 'payroll_pct_change']].transpose()

Payroll Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
payroll_base,"957,243,925,919.46","1,000,927,524,603.83","1,038,549,797,544.50","1,093,118,040,179.03","1,142,184,659,807.88","1,186,943,179,891.72","1,228,896,971,542.67","1,273,459,598,542.40","1,322,147,309,786.19","1,374,230,259,717.77","1,428,746,536,933.71","1,485,099,496,252.99","1,544,084,043,304.97","1,605,765,630,755.24"
payroll_new,"959,890,655,854.82","1,004,437,877,050.48","1,045,836,056,162.13","1,096,294,986,513.55","1,145,564,919,540.03","1,190,722,620,643.34","1,232,243,856,206.53","1,276,910,817,880.97","1,326,248,065,480.81","1,377,457,149,516.46","1,432,531,110,800.36","1,488,338,899,183.01","1,547,715,668,240.92","1,609,168,349,438.11"
payroll_change,"2,646,729,935.36","3,510,352,446.65","7,286,258,617.63","3,176,946,334.52","3,380,259,732.15","3,779,440,751.62","3,346,884,663.86","3,451,219,338.58","4,100,755,694.63","3,226,889,798.69","3,784,573,866.65","3,239,402,930.03","3,631,624,935.95","3,402,718,682.87"
payroll_pct_change,0.28,0.35,0.70,0.29,0.30,0.32,0.27,0.27,0.31,0.23,0.26,0.22,0.24,0.21


In [12]:
def hist_plots(base_data, new_data, year):
    change_wts = new_data['s006'] - base_data['s006']
    pct_change_wts = (change_wts / base_data['s006']) * 100
    hist_diff, edges_diff = np.histogram(change_wts, density=True, bins=50)
    #hist_pct, edges_pct = np.histogram(pct_change_wts, density=True, bins=50)
    
    # create plots
    f1 = figure(title=f'Change in Weights for {year}', width=350, height=350)  # difference
    f1.quad(top=hist_diff, bottom=0, left=edges_diff[:-1], right=edges_diff[1:],
            fill_color='cyan', line_color='black')
    f1.yaxis.axis_label = 'Frequency'
    f1.xaxis.axis_label = 'Change'

    #f2 = figure(title=f'Pct Change in Weights for {year}', width=350, height=350)  # pct change
    #f2.quad(top=hist_pct, bottom=0, left=edges_pct[:-1], right=edges_pct[1:],
    #        fill_color='cyan', line_color='black')
    #f2.yaxis.axis_label = 'Frequency'
    #f2.xaxis.axis_label = 'Pct Change'
    
    return f1#, f2

In [13]:
# analyze change in each record's weight
plot_list_diff = []  # holds graphs showing difference in wt
plot_list_pct = []  # holds graphs showing histogram of pct change in wt
for year in years:
    base_data1 = base_data[year]
    new_data1 = new_data[year]

    #hist_diff1, hist_pct1 = hist_plots(base_data1, new_data1, year)
    hist_diff1 = hist_plots(base_data1, new_data1, year)
    plot_list_diff.append(hist_diff1)
    #plot_list_pct.append(hist_pct1)

In [14]:
show(column(plot_list_diff))

In [15]:
tc.__version__

'0.20.1'

### Raw Files

In [16]:
cur_wts.head()

,WT2011,WT2012,WT2013,WT2014,WT2015,WT2016,WT2017,WT2018,WT2019,WT2020,WT2021,WT2022,WT2023,WT2024,WT2025,WT2026,WT2027
0,126255,193669,202012,211711,218984,226256,235295,243188,253108,261840,270437,281218,290123,300749,311841,322559,330693
1,17482,18243,18900,19543,20215,20886,21576,22300,23057,23853,24636,25451,26256,27041,27858,28631,29353
2,139730,139310,141630,70543,71437,72562,72336,73466,72939,73797,74669,73935,74762,73988,73190,72369,73175
3,27872,27788,28251,28142,28499,28948,29447,29907,30311,30668,31030,31378,31729,32083,32443,32808,33173
4,149520,79008,78808,75485,76442,77646,77404,78613,78049,78968,79901,79115,80000,79172,78319,77440,78302


In [17]:
new_wts.head()

,WT2011,WT2012,WT2013,WT2014,WT2015,WT2016,WT2017,WT2018,WT2019,WT2020,WT2021,WT2022,WT2023,WT2024,WT2025,WT2026,WT2027
0,126255,184446,188363,190540,194165,196088,213479,222251,229795,239445,247308,253648,265472,271456,283675,291543,301016
1,17482,18243,18900,19543,20215,27152,21576,22300,23057,23853,24636,25451,26256,27041,27858,28631,29353
2,139730,139310,141630,91705,95725,101586,93003,92957,94213,93784,94892,97531,95441,98114,95961,97041,96458
3,27872,27788,28251,28142,28499,20263,29447,29907,30311,30668,31030,31378,31729,32083,32443,32808,33173
4,149520,89443,93963,98131,102432,108704,99520,99470,100814,100355,101541,104364,102128,104989,102684,103840,103216


In [18]:
for column in cur_wts.columns:
    cur = cur_wts[column].sum()
    new = new_wts[column].sum()
    print(f'{column}: Current: {cur:,} New: {new:,} Diff: {new - cur:,} Pct: {((new - cur) / cur) * 100:.2f}')

WT2011: Current: 15,454,814,962 New: 15,454,814,962 Diff: 0 Pct: 0.00
WT2012: Current: 15,934,992,640 New: 15,930,314,530 Diff: -4,678,110 Pct: -0.03
WT2013: Current: 16,219,843,516 New: 16,211,230,848 Diff: -8,612,668 Pct: -0.05
WT2014: Current: 16,193,704,375 New: 16,174,046,809 Diff: -19,657,566 Pct: -0.12
WT2015: Current: 16,417,243,644 New: 16,384,528,763 Diff: -32,714,881 Pct: -0.20
WT2016: Current: 16,686,532,466 New: 16,597,289,050 Diff: -89,243,416 Pct: -0.53
WT2017: Current: 16,982,888,408 New: 16,951,537,679 Diff: -31,350,729 Pct: -0.18
WT2018: Current: 17,258,333,089 New: 17,221,775,888 Diff: -36,557,201 Pct: -0.21
WT2019: Current: 17,509,230,586 New: 17,459,131,939 Diff: -50,098,647 Pct: -0.29
WT2020: Current: 17,735,225,806 New: 17,686,839,341 Diff: -48,386,465 Pct: -0.27
WT2021: Current: 17,962,244,861 New: 17,912,756,673 Diff: -49,488,188 Pct: -0.28
WT2022: Current: 18,187,123,527 New: 18,127,772,988 Diff: -59,350,539 Pct: -0.33
WT2023: Current: 18,410,595,147 New: 18,3

### Change in Weight by Tax Liability

In [19]:
def wt_plot(data, year, tax):
    f3 = figure(title=f'Average Change in Weight by Tax Liability - {year}')
    f3.circle(x='perc', y='change', size=5, source=data)
    f3.xaxis.axis_label = 'Tax Liability Percentile'
    f3.yaxis.axis_label = 'Average Weight Change'
    return(f3)

In [20]:
plots = []
data = []
for year in years:
    tax = 'combined'
    temp = pd.DataFrame({'wt_change': new_data[year]['s006'] - base_data[year]['s006'],
                         's006': base_data[year]['s006'],
                         tax: new_data[year][tax]})
    change = percentile(temp, 'wt_change', 100, 'combined', result_type='avg')
    cds = ColumnDataSource({'perc': [i for i in range(1, 101)], 'change': change})
    plot = wt_plot(cds, year, 'Combined')
    plots.append(plot)

In [21]:
for p in plots:
    show(p)

In [22]:
tc.__version__

'0.20.1'